In [1]:
import os
import unittest
import time
import copy
from pymatgen.core.structure import Molecule
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import OpenBabelNN
from pymatgen.util.testing import PymatgenTest
from pymatgen.analysis.reaction_network.reaction_network import ReactionNetwork
from pymatgen.analysis.fragmenter import Fragmenter, metal_edge_extender
from pymatgen.entries.mol_entry import MoleculeEntry
from monty.serialization import loadfn
import openbabel as ob
import networkx as nx

In [2]:
from crystal_toolkit.helpers.pythreejs_renderer import view_old

In [3]:
# prod_entries = []
# entries = loadfn("/Users/samuelblau/Desktop/MgTFSI_entries.json")
# for entry in entries:
#     if "optimized_molecule" in entry["output"]:
#         mol = entry["output"]["optimized_molecule"]
#     else:
#         mol = entry["output"]["initial_molecule"]
#     H = float(entry["output"]["enthalpy"])
#     S = float(entry["output"]["entropy"])
#     E = float(entry["output"]["final_energy"])
#     mol_entry = MoleculeEntry(molecule=mol,energy=E,enthalpy=H,entropy=S,entry_id=entry["task_id"])
#     prod_entries.append(mol_entry)
prod_entries = loadfn("/Users/samuelblau/Desktop/thf_production_mol_entries.json")

In [4]:
RN = ReactionNetwork(prod_entries, electron_free_energy=-2.0)
# RN = loadfn("/Users/samuelblau/Desktop/prod_RN.json")
#
# NOTE: this notebook was made while isomorphism checking was turned off in the reaction network constructor
#

13922 input entries
12790 connected entries
12790 entries not checked for uniqueness
Missing H3O+, will not add concerted water splitting rxn1
Adding concerted water splitting rxn2: 2H2O + 2e- -> H2 + 2OH-
Water rxn2 free energy = 1.3072923625704789


In [6]:
print("MgTFSI +1 potential inds:")
for entry in RN.entries["C2 F6 Mg1 N1 O4 S2"][16][1]:
    print(entry.parameters["ind"])
print()
print("Mg(TFSI)2 0 potential inds:")
for entry in RN.entries["C4 F12 Mg1 N2 O8 S4"][32][0]:
    print(entry.parameters["ind"])
print()
print("Mg(TFSI)(THF) +1 potential inds:")
for entry in RN.entries["C6 F6 H8 Mg1 N1 O5 S2"][30][1]:
    print(entry.parameters["ind"])
print()


MgTFSI +1 potential inds:
4237
4238
4239
4240
4241
4242

Mg(TFSI)2 0 potential inds:
8059

Mg(TFSI)(THF) +1 potential inds:
12281
12282



In [21]:
ind=4237
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
# print(RN.entries_list[ind].entry_id)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-7.3170024603599995, left=-7.3170024603599995, n…

1


-55162.184516301924

In [22]:
ind=4238
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
# print(RN.entries_list[ind].entry_id)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-8.56283845104, left=-8.56283845104, near=-2000.…

1


-55162.219714430394

In [41]:
for neighbor in list(RN.graph.neighbors(4237)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

4237,4234
rxn_type : One electron reduction
bipartite : 1
energy : -0.15066736328003572
free_energy : -2.1196495237891213

4237,4235
rxn_type : One electron reduction
bipartite : 1
energy : -0.15188676324987682
free_energy : -2.1649997522254125

4237,4243
rxn_type : One electron oxidation
bipartite : 1
energy : 0.36065995500007375
free_energy : 7.6796192285983125

4237,4207+12772
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 1.4788755124800446
free_energy : 39.653874015079055

4237,4211+12771
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.5822565273899727
free_energy : 15.337556567101274

4237,4214+12770
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.2648276901600184
free_energy : 6.721877282674541

4237,4219+12769
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.3480959862702093
free_energy : 8.877280117034388

4237,891+1037
rxn_t

In [42]:
ind=4234
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
# print(RN.entries_list[ind].entry_id)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-7.3564274660399995, left=-7.3564274660399995, n…

0


-55166.30416582571

In [66]:
ind=4235
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
print(RN.entries_list[ind].entry_id) 
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-8.49467655864, left=-8.49467655864, near=-2000.…

0
132481


-55166.34951605415

In [153]:
view_old(RN.entries["C2 F6 Mg1 N1 O3 S2"][15][0][0].molecule,bonding_strategy="OpenBabelNN")
RN.entries["C2 F6 Mg1 N1 O3 S2"][15][0][0].entry_id

Renderer(background='white', camera=OrthographicCamera(bottom=-7.065108276599999, left=-7.065108276599999, nea…

127947

In [63]:
for neighbor in list(RN.graph.neighbors(4235)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

4235,4237
rxn_type : One electron oxidation
bipartite : 1
energy : 0.15188676324987682
free_energy : 2.1649997522254125

4235,891+1036
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.7318381054067231
free_energy : 19.174675159258186

4235,890+1037
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.14896775223087388
free_energy : 3.4196352471917635

4235,888+1051
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.11905459124886875
free_energy : 2.5223321792364004

4235,887+1058
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.46527910363988667
free_energy : 11.951146847073687

4235,4207+12771
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.6418273071299154
free_energy : 16.908578652095457

4235,4211+12770
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.27210850883

In [108]:
ind=4215
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
print(RN.entries_list[ind].formula)
print(RN.entries_list[ind].entry_id)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-8.276198036159998, left=-8.276198036159998, nea…

1
C2 F6 Mg1 N1 O3 S2
133982


-53115.05610792261

In [29]:
ind=12281
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
# print(RN.entries_list[ind].entry_id)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-11.17429573848, left=-11.17429573848, near=-200…

1


-61487.16226877529

In [146]:
ind=12282
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
print(RN.entries_list[ind].formula)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-11.38181331348, left=-11.38181331348, near=-200…

1
C6 F6 H8 Mg1 N1 O5 S2


-61487.27956931585

In [156]:
entry = RN.entries["C1 F3 Mg1 N1 O4 S2"][13][0][0]
view_old(entry.molecule,bonding_strategy="OpenBabelNN")
print(entry.parameters["ind"])
print(entry.entry_id)
print(entry.free_energy)

Renderer(background='white', camera=OrthographicCamera(bottom=-5.6305932726, left=-5.6305932726, near=-2000.0,…

1064
125337
-45979.67878623332


In [158]:
entry = RN.entries["C2 F6 Mg1 N1 O4 S2"][16][0][1]
view_old(entry.molecule,bonding_strategy="OpenBabelNN")
print(entry.parameters["ind"])
print(entry.entry_id)
print(entry.free_energy)

Renderer(background='white', camera=OrthographicCamera(bottom=-8.49467655864, left=-8.49467655864, near=-2000.…

4235
132481
-55166.34951605415


In [129]:
RN.entries["C5 F3 H8 Mg1 N1 O5 S2"][26].keys()

dict_keys([2])

In [104]:
ind=12245
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-12.952328526479999, left=-12.952328526479999, n…

2


-50244.11734823951

In [36]:
ind=12280
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-10.99287377868, left=-10.99287377868, near=-200…

0


-61490.39580388127

In [37]:
for neighbor in list(RN.graph.neighbors(12280)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

12280,12281
rxn_type : One electron oxidation
bipartite : 1
energy : 0.11831367307013352
free_energy : 1.233535105980991

12280,12275
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : 0.011568197900032828
free_energy : -0.06380431093566585

12280,887+12253
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.422395002470239
free_energy : 10.749647662894859

12280,4228+11927
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.200100992732132
free_energy : 4.734157420585689

12280,4234+11926
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.035196746316614735
free_energy : 0.39559494215791347

12280,4237+11925
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.19586551265274466
free_energy : 4.76465788538917

12280,4243+11924
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.595257540825969
f

In [38]:
ind=12275
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-10.8507051282, left=-10.8507051282, near=-2000.…

0


-61490.459608192206

In [39]:
for neighbor in list(RN.graph.neighbors(12279)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

12279,12281
rxn_type : One electron oxidation
bipartite : 1
energy : 0.11572478534026232
free_energy : 1.2020828657623497

12279,12275
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : 0.008979310170161625
free_energy : -0.09525655115430709

12279,887+12253
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.4198061147403678
free_energy : 10.718195422676217

12279,4228+11927
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.1975121050022608
free_energy : 4.702705180367047

12279,4234+11926
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.03260785858674353
free_energy : 0.36414270193927223

12279,4237+11925
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.19327662492287345
free_energy : 4.733205645170528

12279,4243+11924
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.59266865309609

In [23]:
for neighbor in list(RN.graph.neighbors(781)):
    if RN.graph.node[neighbor]["rxn_type"] == "Molecular decomposition breaking one bond A -> B+C":
        frag1_id = int(neighbor.split(",")[1].split("+")[0])
        frag2_id = int(neighbor.split(",")[1].split("+")[1])
        frag1 = RN.entries_list[frag1_id]
        frag2 = RN.entries_list[frag2_id]
        #print(neighbor)
        print(RN.graph.node[neighbor]["free_energy"],"eV delta G")
        print(RN.graph.node[neighbor]["energy"]*27.21139,"eV delta E")
        print(frag1.formula,",",frag1.charge)
        print(frag2.formula,",",frag2.charge)
        print()

16.863228423659166 eV delta G
17.431831598816576 eV delta E
C2 F6 Mg1 N1 O3 S2 , -1
O1 , 1

6.8845974497671705 eV delta G
7.371269188215421 eV delta E
C2 F6 Mg1 N1 O3 S2 , 0
O1 , 0

4.261160493704665 eV delta G
4.725831629470079 eV delta E
C2 F6 Mg1 N1 O3 S2 , 1
O1 , -1

12.863120376125153 eV delta G
13.43823475761896 eV delta E
C2 F6 Mg1 N1 O3 S2 , 2
O1 , -2

19.129324930821895 eV delta G
19.88115053493812 eV delta E
C1 F3 , 2
C1 F3 Mg1 N1 O4 S2 , -2

3.3742850187554723 eV delta G
4.0204380352323446 eV delta E
C1 F3 , 1
C1 F3 Mg1 N1 O4 S2 , -1

-0.1190885817995877 eV delta G
0.5449738947088137 eV delta E
C1 F3 , 0
C1 F3 Mg1 N1 O4 S2 , 0

2.476981950800109 eV delta G
3.20645934561822 eV delta E
C1 F3 , -1
C1 F3 Mg1 N1 O4 S2 , 1

11.905796618637396 eV delta G
12.627709579850041 eV delta E
C1 F3 , -2
C1 F3 Mg1 N1 O4 S2 , 2

42.84973000360333 eV delta G
43.42194480119087 eV delta E
C2 F5 Mg1 N1 O4 S2 , -2
F1 , 2

17.66471475733124 eV delta G
18.127455323371073 eV delta E
C2 F5 Mg1 N1 O4 S

In [22]:
for neighbor in list(RN.graph.neighbors(782)):
    if RN.graph.node[neighbor]["rxn_type"] == "Molecular decomposition breaking one bond A -> B+C":
        frag1_id = int(neighbor.split(",")[1].split("+")[0])
        frag2_id = int(neighbor.split(",")[1].split("+")[1])
        frag1 = RN.entries_list[frag1_id]
        frag2 = RN.entries_list[frag2_id]
        #print(neighbor)
        print(RN.graph.node[neighbor]["free_energy"],"eV delta G")
        print(RN.graph.node[neighbor]["energy"]*27.21139,"eV delta E")
        print(frag1.formula,",",frag1.charge)
        print(frag2.formula,",",frag2.charge)
        print()

39.653874015079055 eV delta G
40.24225833154436 eV delta E
C2 F6 Mg1 N1 O3 S2 , -1
O1 , 2

15.337556567101274 eV delta G
15.84400944685423 eV delta E
C2 F6 Mg1 N1 O3 S2 , 0
O1 , 1

6.721877282674541 eV delta G
7.206329559743423 eV delta E
C2 F6 Mg1 N1 O3 S2 , 1
O1 , 0

8.877280117034388 eV delta G
9.472175639833312 eV delta E
C2 F6 Mg1 N1 O3 S2 , 2
O1 , -1

16.626399158965796 eV delta G
17.36564171545293 eV delta E
C1 F3 , 2
C1 F3 Mg1 N1 O4 S2 , -1

2.1274790009338176 eV delta G
2.7387854939241607 eV delta E
C1 F3 , 1
C1 F3 Mg1 N1 O4 S2 , 0

2.309045230074844 eV delta G
2.9832036496150143 eV delta E
C1 F3 , 0
C1 F3 Mg1 N1 O4 S2 , 1

6.279588498407975 eV delta G
7.022496535539472 eV delta E
C1 F3 , -1
C1 F3 Mg1 N1 O4 S2 , 2

40.113316632305214 eV delta G
40.59583833964335 eV delta E
C2 F5 Mg1 N1 O4 S2 , -1
F1 , 2

16.228120385792863 eV delta G
16.667037148922894 eV delta E
C2 F5 Mg1 N1 O4 S2 , 0
F1 , 1

4.846491238567978 eV delta G
5.3148599444106495 eV delta E
C2 F5 Mg1 N1 O4 S2 , 1
F1